In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import torch
import pyarrow.parquet as pq
from textDataset import TranslationDataset, get_and_split_dataset
from model import Transformer, Translator
from utils import load_tokenizer, load_data, build_tokenizer, get_config
from train import train
import numpy as np

In [3]:
config_path = 'config.json'
h = get_config(config_path)
ckpt_path = 'ckpt'

In [4]:
model = Transformer(
    N=h.N, 
    d_model=h.d_model, 
    num_heads=h.num_heads, 
    d_ff=h.d_ff, 
    vocab_size=h.vocab_size, 
    dropout=h.dropout, 
    max_len=h.max_len
).to('cuda')
model.load_state_dict(torch.load('ckpt')['model_state_dict'])

<All keys matched successfully>

In [5]:
data_path = '/home/dhhyun/data/test'
data = load_data(data_path)

1 .parquet files are found..


In [6]:
data[3]

('Diese Frage hat Gutachs Bürgermeister gestern klar beantwortet.',
 "Yesterday, Gutacht's Mayor gave a clear answer to this question.")

In [7]:
tokenizer = load_tokenizer(h.tokenizer_path)

loading tokenizer from file...
done


In [8]:
train_set, valid_set = get_and_split_dataset(data, tokenizer, h.valid_ratio, 'cuda')

length: 3003 / train: 2994


In [9]:
print(train_set[0][0])
print(train_set[0][1].shape)
print(train_set[0][2].shape)

tensor([12828,  3705,    29, 14715,  4178,  5698,  3724,  8003,    74,  6794,
            3], device='cuda:0')
torch.Size([13])
torch.Size([13])


In [10]:
i = 3
pred = model(train_set[i][0].unsqueeze(0), train_set[i][1].unsqueeze(0)).squeeze(0)
ground_truth = train_set[i][2]

In [11]:
pred.shape

torch.Size([18, 15000])

In [12]:
loss_fn = torch.nn.CrossEntropyLoss()

In [13]:
loss = loss_fn(pred, ground_truth)

In [14]:
loss.item()

4.305244445800781

In [15]:
pred_ids = torch.argmax(pred, -1)
print(pred_ids)
tokenizer.decode(pred_ids.tolist())

tensor([5906, 5320,   15, 5272, 3639,   10,   86,   15, 5574, 4660,   68, 4660,
        4660, 3644, 3780, 5272,   17,    3], device='cuda:0')


"These issue , question and ' s , citizens clear a clear clear to this question ."